In [1]:
import redis
import json
import time
import uuid
from gtts import gTTS
import os
from pathlib import Path
from IPython.display import Audio, display

In [2]:
r = redis.Redis()

In [3]:
audio_dir = Path("../public/audio")
audio_dir.mkdir(parents=True, exist_ok=True)

print("Worker started. Listening for tasks on Redis stream 'tts_stream'...")

Worker started. Listening for tasks on Redis stream 'tts_stream'...


In [ ]:
last_id = '0'
while True:
    try:
        entries = r.xread({'tts_stream':last_id}, block=0, count=1)
        for stream, messages in entries:
            for msg_id, message in messages:
                last_id = msg_id.decode()
                task_id = message[b'task_id'].decode()
                text = message[b'text'].decode()

                print(f"Received Task {task_id}: '{text}'")

                r.hset(f'tts_status:{task_id}', mapping={
                    'status': 'processing'
                })

                try: 
                    filename = f"{task_id}.mp3"
                    filepath = audio_dir / filename
                    # tts = gTTS(text)
                    tts = gTTS(text, lang='ur')
                    tts.save(str(filepath))
                    r.hset(f'tts_status:{task_id}', mapping={
                        'status': 'done',
                        'file': f'/audio/{filename}'
                    })
                    print(f"Task {task_id} done. File: {filepath}")
                    display(Audio(str(filepath)))
                except Exception as e:
                    r.hset(f'tts_status:{task_id}', mapping={
                        'status': 'error',
                        'message': str(e)
                    })
                    print(f"Error processing task {task_id}: {e}")
    except KeyboardInterrupt:
        print("worker stopped")
        break

Received Task eab38c53-c6fa-4ca7-bf90-c45f93e7d445: 'انڈیا کے وزیر اعظم نریندر مودی نے کہا ہے کہ ’تجارت اور دہشت گردی ایک ساتھ نہیں ہو سکتے اور نہ ہی پانی اور خون ایک ساتھ نہیں بہہ سکتا ہے۔ اگر پاکستان سے بات ہو گی تو دہشت گردی پر ہی ہو گی‘'
Task eab38c53-c6fa-4ca7-bf90-c45f93e7d445 done. File: ../public/audio/eab38c53-c6fa-4ca7-bf90-c45f93e7d445.mp3
